# Things left to do:

1. Understand how to see tensorboard realtime
2. See if I can make mario render as it is training
3. Figure out load the best model and make it play
4. Figure out how to record and save the best model
5. Make it play in world 1-2
6. Get the regular model to train as well perhaps as a separate model
7. see what to make of the graphs

## Importing packages

In [1]:
#standard packages
import gym
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch as th
from torch import nn
from pathlib import Path
import datetime
from pytz import timezone
import os

# mario packages
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import *

# For custom level
from game.environment import create_gym_env_from_level

# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation

# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

# Import algo
from stable_baselines3 import A2C

# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import VecVideoRecorder, SubprocVecEnv, DummyVecEnv

## Preprocessing & Defining Environment

By default, in each frame the game performs an action (a movement) and returns the reward for that action. What happens, is that to train the AI it is not necessary to make a move in each frame. That is why, the function executes the movement every X frames giving less work to do the training.

In [2]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

By default the enviroment is given by 240*256 pixels. In order to optimize our model it is not necessary to have so many pixels and that is why we can rescale our enviroment to a smaller scale.

In [3]:
class ResizeEnv(gym.ObservationWrapper):
    def __init__(self, env, size):
        gym.ObservationWrapper.__init__(self, env)
        (oldh, oldw, oldc) = env.observation_space.shape
        newshape = (size, size, oldc)
        self.observation_space = gym.spaces.Box(low=0, high=255,
            shape=newshape, dtype=np.uint8)

    def observation(self, frame):
        height, width, _ = self.observation_space.shape
        frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
        if frame.ndim == 2:
            frame = frame[:,:,None]
        return frame

Importing our custom environment function and including all of the processing we define the following function

In [4]:
def create_mario_env(world, stage, version, use_coin_collector_env):
    env = create_gym_env_from_level(world, stage, version, use_coin_collector_env)
    env = JoypadSpace(env, COMPLEX_MOVEMENT)
    env = SkipFrame(env, skip=4)
    env = GrayScaleObservation(env, keep_dim=True)
    env = ResizeEnv(env, size=84)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, 4, channels_order='last')
    return env



In [5]:
# <world> is a number in {1, 2, 3, 4, 5, 6, 7, 8} indicating the world
world = 1
# <stage> is a number in {1, 2, 3, 4} indicating the stage within a world
stage = 1
version = 3
use_coin_collector_env = True

env = create_mario_env(world, stage, version, use_coin_collector_env)

Registering Coin Collector CoinCollectorSuperMarioBrosEnv-1-1-v0 in gym for use later on.
Successfully registered coin collector env CoinCollectorSuperMarioBrosEnv-1-1-v0!
Registering Coin Collector CoinCollectorSuperMarioBrosEnv-1-2-v0 in gym for use later on.
Successfully registered coin collector env CoinCollectorSuperMarioBrosEnv-1-2-v0!
Registering Coin Collector CoinCollectorSuperMarioBrosEnv-1-3-v0 in gym for use later on.
Successfully registered coin collector env CoinCollectorSuperMarioBrosEnv-1-3-v0!
Registering Coin Collector CoinCollectorSuperMarioBrosEnv-1-4-v0 in gym for use later on.
Successfully registered coin collector env CoinCollectorSuperMarioBrosEnv-1-4-v0!
Registering Coin Collector CoinCollectorSuperMarioBrosEnv-2-1-v0 in gym for use later on.
Successfully registered coin collector env CoinCollectorSuperMarioBrosEnv-2-1-v0!
Registering Coin Collector CoinCollectorSuperMarioBrosEnv-2-2-v0 in gym for use later on.
Successfully registered coin collector env CoinCol

In [6]:
env.reset()
state, reward, done, info = env.step([0])
print('state:', state.shape) #Color scale, height, width, num of stacks

state: (1, 84, 84, 4)


## Model Training

Model parameters

Model

In [7]:
class MarioNet(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim):
        super(MarioNet, self).__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(th.as_tensor(observation_space.sample()[None]).float()).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=MarioNet,
    features_extractor_kwargs=dict(features_dim=512),
)

The next step consists of the creation of a file where the AI will save the results obtained in each iteration. In this way, later we will be able to visualize graphically the learning of our model.

In this case, the average score, the average starting time and the best score obtained will be saved for each iteration.

In [23]:
# Get the absolute path of the current directory
current_dir = os.path.abspath(os.path.dirname("."))

# Define the directory name
dir_name = 'model'

# Combine the current directory path with the directory name
save_dir = os.path.join(current_dir, dir_name)

# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [24]:
save_dir

'C:\\Parashara\\Projects\\NUS projects\\Sem2\\its_ah_me_mario\\agents\\actorcritic\\model'

In [1]:
reward_log_path = os.path.join(save_dir,'reward_log.csv')

with open(reward_log_path, 'a') as f:
    print('timesteps,reward,best_reward', file=f)

NameError: name 'os' is not defined

This callback function will be in charge of writing the aforementioned data to the file. This function will be executed automatically each time an iteration has been completed.

In [27]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(save_dir, f'best_model_{self.n_calls}')
            self.model.save(model_path)

            total_reward = [0] * EPISODE_NUMBERS
            total_time = [0] * EPISODE_NUMBERS
            best_reward = 0

            for i in range(EPISODE_NUMBERS):
                state = env.reset()  # reset for each new trial
                done = False
                total_reward[i] = 0
                total_time[i] = 0
                while not done and total_time[i] < MAX_TIMESTEP_TEST:
                    action, _ = model.predict(state)
                    state, reward, done, info = env.step(action)

                    # This should render it
                    # env.render()

                    total_reward[i] += reward[0]
                    total_time[i] += 1

                if total_reward[i] > best_reward:
                    best_reward = total_reward[i]
                    best_epoch = self.n_calls

                state = env.reset()  # reset for each new trial

            print('time steps:', self.n_calls, '/', TOTAL_TIMESTEP_NUMB)
            print('average reward:', (sum(total_reward) / EPISODE_NUMBERS),
                  'average time:', (sum(total_time) / EPISODE_NUMBERS),
                  'best_reward:', best_reward)

            with open(reward_log_path, 'a') as f:
                print(self.n_calls, ',', sum(total_reward) / EPISODE_NUMBERS, ',', best_reward, file=f)

        return True

In [29]:
# Model Param
# CHECK_FREQ_NUMB = 10000
CHECK_FREQ_NUMB = 1000
TOTAL_TIMESTEP_NUMB = 5000000
LEARNING_RATE = 0.0001
GAE = 1.0
ENT_COEF = 0.01
N_STEPS = 512
GAMMA = 0.9
BATCH_SIZE = 64
N_EPOCHS = 10

# Test Param
EPISODE_NUMBERS = 20
MAX_TIMESTEP_TEST = 1000

In [30]:
callback = TrainAndLoggingCallback(check_freq=CHECK_FREQ_NUMB, save_path=save_dir)

## Model Training

In [37]:
model = A2C('CnnPolicy', env, verbose=2, policy_kwargs=policy_kwargs, tensorboard_log=save_dir,
            learning_rate=LEARNING_RATE, n_steps=N_STEPS, gamma=GAMMA, gae_lambda=GAE,
            ent_coef=ENT_COEF)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [36]:
model.learn(total_timesteps=TOTAL_TIMESTEP_NUMB, callback=callback, log_interval=1)

AttributeError: 'NoneType' object has no attribute 'reset'

## Model testing

To get the best epoch we would need to check the graph and see the best epoch from there

In [33]:
best_epoch = 5000

In [34]:
best_model_path = os.path.join(save_dir, 'best_model_{}'.format(best_epoch))
model = A2C.load(best_model_path)


In [35]:
state = env.reset()
done = True
plays = 0;
wins = 0;
while plays < 100:
    if done:
        state = env.reset()
        if info[0]["flag_get"]:
          wins += 1
        plays += 1
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()
print("Model win rate: " + str(wins) + "%")

C:\Users\paras\anaconda3\envs\mario-baselines\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))

KeyboardInterrupt

